# Once Auto3D has completed, run this notebook to compare the RMSD of generated samples and the optimised geometries

Notebook outputs `data_optim.csv` with new RMSD results

In [13]:
#When the experiment finishes, the data will be here
#"geometry_stability/results/results_smiles/smiles_out.sdf"

In [14]:
#auto_3d_results_path = "geometry_stability/benchmark_100_smiles/smiles_out.sdf"
experiment_path = "experiments/"

In [15]:
#While the experiment is still running, use this
import glob

#results_fast
#results
#auto_3d_results_paths = glob.glob("geometry_stability/results_fast/results_fast_smiles/*/*3d.sdf")
#auto_3d_results_paths

auto_3d_results_path = "geometry_stability/results_fast/results_fast_smiles/smiles_out.sdf"

auto_3d_results_paths = [auto_3d_results_path]

# This should really be in "util"

In [16]:
from rdkit.Chem import Descriptors
from rdkit.Chem.MolStandardize import rdMolStandardize
from openbabel import pybel
from rdkit import Chem


def rdkit_fix_radicals(mol,add_h=False,flatten=False,uncharge=True):
    """
    Atoms with unfilled valance get radicals assigned.
    Openbabel will have assigned bond orders based on bond length.
    Here I assume all radical electrons should instead be hydrogen atoms
    """
    for atom in mol.GetAtoms():
        radicals = atom.GetNumRadicalElectrons()
        atom.SetNumRadicalElectrons(0)
        atom.SetNumExplicitHs(atom.GetNumExplicitHs() + radicals)

    if flatten:
        mol = Chem.MolFromSmiles(Chem.MolToSmiles(mol))

    if add_h:
        mol = Chem.AddHs(mol,addCoords=True)

    if uncharge:
        un = rdMolStandardize.Uncharger()
        mol = un.uncharge(mol)

    return mol


def xyz_to_mol_clean(xyz, add_h=True, flatten=False):
    """
    add_h - add RDKit hydrogens
    flatten - run Chem.MolFromSmiles(Chem.MolToSmiles(x)) such that geometry infromation is lost
    
    Sometimes these imports fail
    In these cases this function returns False
    """
    try:
        mol_pybel = pybel.readstring("xyz", xyz)
        mol_mol2 = mol_pybel.write("mol2")

        #RDKit - clean radicals
        mol_rdkit = Chem.MolFromMol2Block(mol_mol2)
        mol_final = rdkit_fix_radicals(mol_rdkit, add_h=add_h, flatten=flatten)

        return mol_final
    
    except:
        return False
    
    
def get_mol_id_from_sdf(mol_sdf):
    string_id = mol_sdf.GetPropsAsDict()["ID"]
    string_id = string_id.split("_")[0]
    split_string_id = string_id.split("-")

    exp_id = "_".join(split_string_id[:2])
    mol_id = "_".join(split_string_id[2:])

    return exp_id, mol_id

In [17]:
from rdkit.Chem import rdMolAlign
from rdkit import Chem
import numpy as np

results = []
for auto_3d_results_path in auto_3d_results_paths:
    suppl = Chem.SDMolSupplier(auto_3d_results_path)

    
    for optimised_mol in suppl:
        exp_id, mol_id = get_mol_id_from_sdf(optimised_mol)
        with open(f"experiments/{exp_id}/{mol_id}.txt", "r") as readfile:
            sample_xyz = readfile.read()
            sample_mol = xyz_to_mol_clean(sample_xyz)
        try:
            rmsd = rdMolAlign.AlignMol(optimised_mol, sample_mol)
        except:
            print("ERROR")
            rmsd = np.NaN

        results.append([mol_id, rmsd])

[17:48:12] Running Uncharger
[17:48:12] Removed negative charge.
*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is stable:False satoms:23 tatoms:31 sratio:0.7419354838709677)

[17:48:12] Running Uncharger
[17:48:12] Removed negative charge.
[17:48:12] Running Uncharger
[17:48:12] Running Uncharger
[17:48:12] Removed negative charge.
[17:48:12] Removed negative charge.
[17:48:12] Running Uncharger
[17:48:12] Running Uncharger
[17:48:12] Removed negative charge.
[17:48:12] Running Uncharger
[17:48:12] Removed negative charge.
[17:48:12] Running Uncharger
[17:48:12] Removed negative charge.
[17:48:12] Removed negative charge.
[17:48:12] Removed negative charge.
[17:48:12] Removed negative charge.
[17:48:12] Running Uncharger
[17:48:12] Running Uncharger
[17:48:12] Removed negative charge.
[17:48:12] Running Uncharger
[17:48:12] Removed negative charge.
[17:48:12] Removed negative charge.
[17:48:12] Running Uncharger
[1

[17:48:12] Running Uncharger
[17:48:12] Removed negative charge.
[17:48:12] Removed negative charge.
*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is stable:False satoms:18 tatoms:31 sratio:0.5806451612903226)

[17:48:12] Running Uncharger
[17:48:12] Removed negative charge.
[17:48:12] Removed negative charge.
[17:48:12] Running Uncharger
[17:48:12] Removed negative charge.
[17:48:12] Removed negative charge.
[17:48:12] Removed negative charge.
*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is stable:False satoms:21 tatoms:31 sratio:0.6774193548387096)

[17:48:12] Running Uncharger
[17:48:12] Removed negative charge.
[17:48:12] Running Uncharger
[17:48:12] Removed negative charge.
[17:48:12] Removed negative charge.
[17:48:12] Removed negative charge.
[17:48:12] Running Uncharger
[17:48:12] Removed negative charge.
[17:48:12] Removed negative ch

[17:48:12] stable:False satoms:4 tatoms:31 sratio:0.12903225806451613: Warning - no explicit hydrogens in mol2 file but needed for formal charge estimation.
[17:48:12] Running Uncharger
[17:48:12] Removed negative charge.
[17:48:12] Running Uncharger
[17:48:12] Removed negative charge.
[17:48:12] Removed negative charge.
[17:48:12] Removed negative charge.
[17:48:12] Removed negative charge.
[17:48:12] Running Uncharger
[17:48:12] Running Uncharger
[17:48:12] Removed negative charge.
[17:48:12] Removed negative charge.
*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is stable:False satoms:5 tatoms:31 sratio:0.16129032258064516)

[17:48:12] Running Uncharger
*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is stable:False satoms:7 tatoms:31 sratio:0.22580645161290322)

[17:48:12] Running Uncharger
[17:48:12] Removed negative charge.
[17:48:12] Remove

[17:48:13] stable:False satoms:4 tatoms:31 sratio:0.12903225806451613: Warning - no explicit hydrogens in mol2 file but needed for formal charge estimation.
[17:48:13] Running Uncharger
[17:48:13] Removed negative charge.
[17:48:13] Removed negative charge.
[17:48:13] stable:False satoms:5 tatoms:31 sratio:0.16129032258064516: Warning - no explicit hydrogens in mol2 file but needed for formal charge estimation.
[17:48:13] Running Uncharger
[17:48:13] Removed negative charge.
[17:48:13] Removed negative charge.
[17:48:13] stable:False satoms:4 tatoms:31 sratio:0.12903225806451613: Warning - no explicit hydrogens in mol2 file but needed for formal charge estimation.
[17:48:13] Running Uncharger
[17:48:13] stable:False satoms:6 tatoms:31 sratio:0.1935483870967742: Warning - no explicit hydrogens in mol2 file but needed for formal charge estimation.
[17:48:13] Running Uncharger
[17:48:13] Removed negative charge.
[17:48:13] Removed negative charge.
[17:48:13] stable:False satoms:6 tatoms:3

[17:48:13] Running Uncharger
[17:48:13] Running Uncharger
*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is stable:False satoms:26 tatoms:31 sratio:0.8387096774193549)

[17:48:13] Running Uncharger
[17:48:13] Running Uncharger
[17:48:13] Running Uncharger
[17:48:13] Removed negative charge.
[17:48:13] Removed negative charge.
[17:48:13] Running Uncharger
[17:48:13] Removed negative charge.
[17:48:13] Removed negative charge.
[17:48:13] Removed negative charge.
[17:48:13] Removed negative charge.
[17:48:13] Removed negative charge.
[17:48:13] Running Uncharger
[17:48:13] Removed negative charge.
[17:48:13] Running Uncharger
*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is stable:False satoms:20 tatoms:31 sratio:0.6451612903225806)

[17:48:13] Running Uncharger
[17:48:13] Removed negative charge.
[17:48:13] Removed negative charge.
[17:48:13] Runn

...

[17:48:21] Running Uncharger
*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is stable:False satoms:17 tatoms:31 sratio:0.5483870967741935)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is stable:False satoms:20 tatoms:31 sratio:0.6451612903225806)

[17:48:21] Running Uncharger
[17:48:21] Removed negative charge.
[17:48:21] Removed negative charge.
[17:48:21] Removed negative charge.
[17:48:21] Removed negative charge.
[17:48:21] Removed negative charge.
[17:48:21] stable:False satoms:4 tatoms:31 sratio:0.12903225806451613: Warning - no explicit hydrogens in mol2 file but needed for formal charge estimation.
[17:48:21] Running Uncharger
*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is stable:False satoms:16 tatoms:31 sratio:0.5161290322580645)

[17:48:21] Running Uncharger
[

[17:48:21] stable:False satoms:6 tatoms:31 sratio:0.1935483870967742: Warning - no explicit hydrogens in mol2 file but needed for formal charge estimation.
[17:48:21] Running Uncharger
[17:48:21] Removed negative charge.
[17:48:21] Removed negative charge.
[17:48:21] stable:False satoms:3 tatoms:31 sratio:0.0967741935483871: Warning - no explicit hydrogens in mol2 file but needed for formal charge estimation.
[17:48:21] Running Uncharger
[17:48:21] Removed negative charge.
[17:48:21] Removed negative charge.
[17:48:21] stable:False satoms:5 tatoms:31 sratio:0.16129032258064516: Warning - no explicit hydrogens in mol2 file but needed for formal charge estimation.
[17:48:21] Running Uncharger
[17:48:21] Removed negative charge.
[17:48:21] stable:False satoms:4 tatoms:31 sratio:0.12903225806451613: Warning - no explicit hydrogens in mol2 file but needed for formal charge estimation.
[17:48:21] Running Uncharger
[17:48:21] stable:False satoms:5 tatoms:31 sratio:0.16129032258064516: Warning

[17:48:21] Running Uncharger
[17:48:21] Removed negative charge.
[17:48:21] Removed negative charge.
[17:48:21] Removed negative charge.
[17:48:21] Removed negative charge.
[17:48:21] Running Uncharger
[17:48:21] Running Uncharger
[17:48:21] Removed negative charge.
[17:48:21] Running Uncharger
[17:48:21] Removed negative charge.
[17:48:21] Running Uncharger
[17:48:21] Running Uncharger
*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is stable:False satoms:13 tatoms:31 sratio:0.41935483870967744)

[17:48:21] Running Uncharger
[17:48:21] Running Uncharger
[17:48:21] Removed negative charge.
[17:48:21] Running Uncharger
[17:48:21] Removed negative charge.
[17:48:21] Removed negative charge.
[17:48:21] Removed negative charge.
[17:48:21] Removed negative charge.
[17:48:21] Running Uncharger
[17:48:21] Removed negative charge.
[17:48:21] Removed negative charge.
[17:48:21] Removed negative charge.
[17:48:21] Running Unch

[17:48:22] Running Uncharger
[17:48:22] Removed negative charge.
*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is stable:False satoms:15 tatoms:31 sratio:0.4838709677419355)

[17:48:22] Running Uncharger
[17:48:22] Removed negative charge.
*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is stable:False satoms:17 tatoms:31 sratio:0.5483870967741935)

[17:48:22] Running Uncharger
[17:48:22] Removed negative charge.
[17:48:22] Removed negative charge.
*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is stable:False satoms:19 tatoms:31 sratio:0.6129032258064516)

[17:48:22] Running Uncharger
[17:48:22] Removed negative charge.
[17:48:22] Removed negative charge.
[17:48:22] Running Uncharger
[17:48:22] Removed negative charge.
[17:48:22] Removed negative charge.
[17:48:22] Removed ne

[17:48:22] stable:False satoms:2 tatoms:31 sratio:0.06451612903225806: Warning - no explicit hydrogens in mol2 file but needed for formal charge estimation.
[17:48:22] Running Uncharger
[17:48:22] stable:False satoms:7 tatoms:31 sratio:0.22580645161290322: Warning - no explicit hydrogens in mol2 file but needed for formal charge estimation.
[17:48:22] Running Uncharger
[17:48:22] Removed negative charge.
[17:48:22] Removed negative charge.
[17:48:22] stable:False satoms:3 tatoms:31 sratio:0.0967741935483871: Warning - no explicit hydrogens in mol2 file but needed for formal charge estimation.
[17:48:22] Running Uncharger
[17:48:22] stable:False satoms:6 tatoms:31 sratio:0.1935483870967742: Warning - no explicit hydrogens in mol2 file but needed for formal charge estimation.
[17:48:22] Running Uncharger
[17:48:22] Removed negative charge.
[17:48:22] Removed negative charge.
[17:48:22] stable:False satoms:6 tatoms:31 sratio:0.1935483870967742: Warning - no explicit hydrogens in mol2 file

# Merge optimisation data with data.csv. Make new file data_optim.csv

In [18]:
import pandas as pd

old_df = pd.read_csv("experiments/data.csv")
rmsd_df = pd.DataFrame(results, columns=["mol_id", "auto3d_rmsd"]).set_index("mol_id")#Note this was rmsd
df = pd.merge(old_df,rmsd_df, on="mol_id",how='left')

df.to_csv(experiment_path+"data_optim.csv",index=False)